## Dependencies

In [88]:
# Confirmed needed dependencies
import random
import pprint
import pandas as pd
import requests
import json

# Dependencies for geocoordinates generator
import sys
import math
import gmplot

# Dependencies for conversion of coordinates to addresses
import geopy
from geopy.geocoders import Nominatim

# Dependencies for Zillow data
from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults
# zillow id, needed for the GetUpdatedPropertyDetails
from pyzillow.pyzillow import ZillowWrapper, GetUpdatedPropertyDetails


# Research how to make config.py truly invisible on GitHub Repo
from config import Zseth, Zkat, Zyuta
# Paste in your zwsid in the meantime using the following line
# zwsid = ""



# In README give instructions on setting config file and needed variables for project

## File inputs/outputs

In [15]:
randLatLon_csv = "./Data/randomLatLon.csv" 
addressList_csv = "./Data/addressList.csv"
masterData_csv = "./Data/masterData.csv"

In [3]:
# Valerie

# Further setting up Notebook with Markdown sections, file outputs, action comments, etc.
# Create pandas dataframe 

## Geocoordinates of Austin

In [4]:
##########################################
# this section written by troy bailey.   #
# enter uservariables below to determine #
# center location, radius of circle, and #
# number of geocoordinates to generate.  #
##########################################

In [5]:
########################
#### USER VARIABLES ####
########################

x0 = 30.27444       #### Set center coordiantes in decimal degrees
y0 = -97.74028      #### initial coordiantes are location of Texas State Capitol Building

radius = 10         #### Set radius in miles

points = 1000        #### Set number of lat,lon points to generate

In [6]:
# variables and inputs for coordinate calculations
lat_lon_list = []
radiusInDegrees=radius/69           
r = radiusInDegrees
points += 1

In [7]:
# calculate each coordiante point and build a list of lat and lon
for i in range(1, points):
    u = float(random.uniform(0.0,1.0)) #random number for radius length
    v = float(random.uniform(0.0,1.0)) #random number for pi radians
    
    w = r * math.sqrt(u) #radius length
    t = 2 * math.pi * v  #radians
    x = w * math.cos(t)  #calculate x coord distance
    y = w * math.sin(t)  #calculate y coord distance
    
    xLat  = x + x0       #offset x by center x
    yLon = y + y0        #offset y by center y
    
    lat_lon_list.append([xLat,yLon])

# convert list to dataframe
lat_lon_df = pd.DataFrame(lat_lon_list, columns=['lat','lon'])

lat_lon_df.head()

len(lat_lon_df)

1000

In [8]:
# write a CSV file of coordinate points
lat_lon_df.to_csv(randLatLon_csv, index=False, header=True)

## Plot coordinate points on map


In [9]:
# This section will plot points on a Google map centered at centerPointLat and centerPointLon with a magnification of magFactor
# It assumes there is a dataframe with "lat" and "lon" columns
# The resulting map is saved to a file called 

centerPointLat = 30.27444  #these are the coordinates of the Texas State Capitol building
centerPointLon = -97.74028 #these are the coordinates of the Texas State Capitol building
magnificationFactor = 10
pointColor = "red"
pointSize = 100
mapOutputFile = "mymap.html"
df = lat_lon_df

gmap = gmplot.GoogleMapPlotter(centerPointLat, centerPointLon, magnificationFactor)

gmap.scatter(df["lat"], df["lon"], pointColor, size=pointSize, marker=False)

gmap.draw("./Visuals/" + mapOutputFile)

## Convert Coordinates to Residential Addresses

In [ ]:
#########################
##### Yuta's Blocks #####
#########################


##### General Analysis on the Geopy Nominatim Method Below #####
# - You can filter the address by type='house', but some houses are labeled with type='yes'
# - 'Yes' type means means it's an unlabeled building.
# - Quite a bit of 'yes' type buildings actually hit a residential building, so we may get a high percentage of
#   residential buildings if we filter by 'house' or 'yes'.
# - Out of roughly 100 requests, around 70% will result in either 'house' or 'yes' type.
# - I can't seem to find the limit of API request, but we are throttled to 1 request per second.

# Testing Geopy Nominatim API Response


geopy.geocoders.options.default_user_agent = "my-application"

pp = pprint.PrettyPrinter(indent=4)

url = "https://nominatim.openstreetmap.org/reverse?"

# Params 1 - Known Residential Address
params_1 = {
    "format": "jsonv2",
    "lat": 30.440777,
    "lon": -97.777048
}

print("===== #01 - Home Response:")
response_1 = requests.get(url, params=params_1).json()
pp.pprint(response_1)
print("\n" + "="*60 + "\n")

In [ ]:
# Import CSV, put into DataFrame
latlon_df = pd.read_csv(randLatLon_csv)
latlon_df.head()

In [ ]:
# Put latitudes and longitudes into a zip object
lats = latlon_df.iloc[:, 0]
lons = latlon_df.iloc[:, 1]
lat_lons = []
lat_lons = zip(lats, lons)

# Might not need to do this step, see below

In [ ]:
# Loop Request API and append to create new dataframe
query_url = "https://nominatim.openstreetmap.org/reverse?"

house_num = []
road = []
postcode = []
# Verify this field is unneeded, then delete if not
# aType = []
lat = []
lon = []
neighborhood = []
# Verify this field is unneeded, then delete if not
# addType = []

counter = 1
numRequests = len(lats)

print(f"Processing {numRequests} Requests...")


# Don't actually need to zip lat and lons for this
# can use "for lat_lon in latlon_df.itertuples():"
# Check into this and see if you can cut out the zip part
for lat_lon in lat_lons:
    params = {
        "format": "jsonv2",
        "lat": lat_lon[0],
        "lon": lat_lon[1]
    }
# Check to see if you can do multiple queries at the same time to make this faster
# i.e. send batch requests/list of queries rather than loop of single queries
    response = requests.get(query_url, params=params).json()
    
    if response['type'] == 'house' or response['type'] == 'yes':
        postcode.append(response['address']['postcode'])
#         Do we need to store aType?
#         aType.append(response['type'])
        lat.append(response['lat'])
        lon.append(response['lon'])
    
        try:
            house_num.append(response['address']['house_number'])
        except (KeyError, IndexError):
            house_num.append("NA")
        try:
            road.append(response['address']['road'])
        except (KeyError, IndexError):
            road.append("NA")
        try:
            neighborhood.append(response['address']['neighbourhood'])
        except (KeyError, IndexError):
            neighborhood.append("NA")
            
# Verify this field is unneeded, then delete if not
#         try:
#             addType.append(response['addresstype'])
#         except (KeyError, IndexError):
#             addType.append("NA")


# Printing to console is good for development to see what is happening, but takes a long time.  
# Instead of printing each line to the console, can you store these in two lists (success and failure) 
# that we could run len() on to get counts for stats later?
        print(f"Processed Record {counter} of {numRequests}.")
#         counter = counter + 1
        counter += 1
    else:
        print(f"Wrong Type - Skipped Record {counter} of {numRequests}.")
#         counter = counter + 1
        counter += 1

In [14]:
# Create a dataframe with all addresses from API requests
address_df = pd.DataFrame({
    "house #": house_num,
    "street": road,
    "zipcode": postcode,
#     "type": aType,
    "lat": lat,
    "lon": lon,
    "neighborhood": neighborhood,
#     "address type": addType
})
address_df

# drop type and address type, or rather don't gather in the first place

NameError: name 'house_num' is not defined

In [ ]:
# write a CSV file of addresses
address_df.to_csv(addressList_csv, index=False, header=True)

In [20]:
# Pull from CSV
address_sample = pd.read_csv(addressList_csv)
address_sample.head()

,house #,street,zipcode,lat,lon,neighborhood
0,4704,Carter Lane,78744,30.205996,-97.735289,Southeast Austin
1,4804,Precipice Cove,78731,30.341652,-97.785410,Cat Mountain
2,2110,West Slaughter Lane,78748,30.175305,-97.825010,Ford Oaks
3,8920,Ramirez Lane,78742,30.245791,-97.647754,NaN
4,11266,Taylor Draper Lane,78759,30.410477,-97.752934,Angus Valley


In [17]:
# Map out the sample with gmplot

gmap = gmplot.GoogleMapPlotter(30.27444, -97.74028, 10)

gmap.scatter(address_sample["lat"], address_sample["lon"], 'red', size=80, marker=False)

gmap.draw("./Visuals/myaddressmap.html")

## Zillow API Calls using Address and Zipcode

In [ ]:
# Val create function for read in csv, check for headers?, and append csv
address_sample = pd.read_csv(addressList_csv)
address_df = pd.DataFrame(address_sample)
address_df.head()
# masterData.csv
address = zip(address_df["house #"], address_df["street"])
len(address_df)

In [76]:
################# NO LONGER USING THIS API #################
########### HOWEVER, CODE MAY BE USED ELSEWHERE ###########

# pp = pprint.PrettyPrinter(indent=4)
# api = zillow.ValuationApi()
# # Insert your Zillow API key here
# zwsid = "X1-ZWz1gm14kn7d3f_2kpit"

# valuation = []
# sqft = []

# for row, home in address_df.iterrows():
#     print(row)
#     address = address_df["house #"][row] + " " + address_df["street"][row] 
#     print(address)
#     zipcode = address_df["zipcode"][row]
#     print(zipcode)
    
#     try:
#         z_deep_results = api.GetDeepSearchResults(zwsid, address, zipcode)
#         print(z_deep_results)
#         pp.pprint(deep_results.get_dict())
#         print(z_deep_results['zestimate']['amount'])
#         valuation.append(z_deep_results['zestimate']['amount'])
#         print(z_deep_results['extended_data']['finished_sqft'])
#         sqft.append(z_deep_results['extended_data']['finished_sqft'])
#     except:
#         print(f"No record found for {address}, {zipcode}.")



In [90]:
address = "4804 Precipice Cove"
zipcode = "78731"

zid = []
valuation = []
sqft = []
isd = []


##### VALERIE STILL WORKING ON THE LOOPING FUNCTION OF THIS #####
###### HOWEVER, ZILLOW ONLY ALLOWS 1000 API CALLS PER DAY ######

# for row, home in address_df.iterrows():
#     print(row)
#     print(home)
# address = str(address_df["house #"][1] + " " + address_df["street"][1] )
# print(address)
# zipcode = str(address_df["zipcode"][1])
# print(zipcode)

# print(f"No record found for {address}, {zipcode} (index {row}).")

zillow_data = ZillowWrapper(Zyuta)
deep_search_response = zillow_data.get_deep_search_results(address, zipcode)
result = GetDeepSearchResults(deep_search_response)


try:
    zillowID = result.zillow_id
    zid.append(result.zillow_id)
except:
    zid.append(None)

try:    
    val = result.zestimate_amount
    valuation.append(val)
except:
    valuation.append(None)

try:
    zsqft = result.home_size
    sqft.append(zsqft)
except:
    sqft.append(None)

updated_property_details_response = zillow_data.get_updated_property_details(zillowID)
new_result = GetUpdatedPropertyDetails(updated_property_details_response)

try:
    zisd = new_result.school_district
    isd.append(zisd)
except:
    isd.append(None)

In [91]:
print(valuation)

['2507628']


In [7]:
# Do not run yet

# masterDF = pd.DataFrame({
#     "address": house_num,
#     "zipcode": postcode,
#     "alat": lat,
#     "alon": lon,
#     "neighborhood": neighborhood,
# })

[]


## Calculate Value per Sqft

In [ ]:
#Create function to calculate value per sqft using "zestimate"["amount"] / "extended_data"["finished_sqft"]


## Crime Data

In [ ]:
# Kat's section

## School Data

In [ ]:
# Seth's section